In [1]:
# %pip install tensorflow

In [2]:
import tensorflow as tf
tf.device('/cpu:0')

In [3]:
TRAIN_FILE_PATH = './Data/TFR/Training_demo.tfrecord.gz'
TEST_FILE_PATH = './Data/TFR/Validation_demo.tfrecord.gz'
LABEL = 'Class'           # name of the label band
BANDS = ['B2', 'B3', 'B4', 'B8']
# Number of label values, i.e. number of classes in the classification.
N_CLASSES = 4
# These names are used to specify properties in the export of
# training/testing data and to define the mapping between names and data
# when reading into TensorFlow datasets.
FEATURE_NAMES = BANDS + [LABEL]
columns = [
  tf.io.FixedLenFeature(shape=[1], dtype=tf.float32) for k in FEATURE_NAMES
]
# Dictionary with feature names as keys, fixed-length features as values.
FEATURES_DICT = dict(zip(FEATURE_NAMES, columns))

In [4]:
# Create a dataset from the TFRecord file in Cloud Storage.
train_dataset = tf.data.TFRecordDataset(
    [TRAIN_FILE_PATH, TEST_FILE_PATH], compression_type='GZIP')

In [5]:
def parse_tfrecord(example_proto):
  """Deserialize an example proto.  Reshape to (1, 1, 1)."""
  parsed_features = tf.io.parse_example(example_proto, FEATURES_DICT)
  labels = parsed_features.pop(LABEL)
  # (1) -> (1, 1, 1)
  return ({k: [[v]] for k, v in parsed_features.items()},
          tf.expand_dims(tf.expand_dims(
              tf.cast(labels, tf.int32), axis=0), axis=0))

# Map the function over the dataset.
parsed_dataset = train_dataset.map(parse_tfrecord, num_parallel_calls=4)

In [8]:
for features, labels in parsed_dataset.take(1):
  print(features)
  print(labels)